# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [2]:
# create a Spark session
spark = SparkSession.builder.appName("Sparkify").getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
df_sparkify = spark.read.json("mini_sparkify_event_data.json")

In [4]:
df_sparkify.count()

286500

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [5]:
df_sparkify.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
df_sparkify.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [7]:
df_sparkify.select("page").dropDuplicates().sort("page").show()

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|               Login|
|              Logout|
|            NextSong|
|            Register|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
| Submit Registration|
|      Submit Upgrade|
|         Thumbs Down|
+--------------------+
only showing top 20 rows



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [9]:
df_sparkify.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [10]:
df_sparkify.createOrReplaceTempView("user_log_table")

In [11]:
spark.sql("Select count(distinct userId) from user_log_table where auth = 'Cancelled'").show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                    52|
+----------------------+



In [12]:
spark.sql("Select count(distinct userId) from user_log_table").show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   226|
+----------------------+



In [13]:
df_label = spark.sql('''
          Select userId, 
          Max(case when auth = 'Cancelled' then 1 else 0 end) as Churn 
          From user_log_table
          Group by userId
          ''')

In [14]:
df_feature = spark.sql('''
Select 
label.userId, 
label.Churn,
COALESCE (thumsup.ThumbsUp,0) As ThumbsUp,
COALESCE (thumsdown.ThumbsDown,0) As ThumbsDown,
COALESCE (song.SongPlayed,0) As SongPlayed,
COALESCE (day.Days,0) As Days,
COALESCE (Round(song.SongPlayed/(day.Days*24),2),0) As SongPerHour
From
(Select userId, 
Max(case when auth = 'Cancelled' then 1 else 0 end) as Churn 
From user_log_table
Group by userId) label

Left Join          
(Select userId,
Count(page) As ThumbsUp
From user_log_table
Where page = 'Thumbs Up'
Group by userId) thumsup
On label.userId = thumsup.userId

Left Join          
(Select userId,
Count(page) As ThumbsDown
From user_log_table
Where page = 'Thumbs Down'
Group by userId) thumsdown
On label.userId = thumsdown.userId

Left Join          
(Select userId,
Count(song) As SongPlayed
From user_log_table
Group by userId) song
On label.userId = song.userId

Left Join          
(Select userId,
Round((Max(ts) - Min(ts))/86400000) As Days
From user_log_table
Group by userId) day
On label.userId = day.userId
''')

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [15]:
assembler = VectorAssembler(inputCols=["SongPlayed", "ThumbsUp", "ThumbsDown", "Days", "SongPerHour"], outputCol="FeatureVector")
df_feature = assembler.transform(df_feature)

In [16]:
scaler = StandardScaler(inputCol="FeatureVector", outputCol="ScaledFeatures", withStd=True)
scaler_model = scaler.fit(df_feature)
df_feature = scaler_model.transform(df_feature)

In [43]:
df_feature.show()

+------+-----+--------+----------+----------+----+-----------+--------------------+--------------------+
|userId|Churn|ThumbsUp|ThumbsDown|SongPlayed|Days|SongPerHour|       FeatureVector|      ScaledFeatures|
+------+-----+--------+----------+----------+----+-----------+--------------------+--------------------+
|100010|    0|      17|         5|       275|44.0|       0.26|[275.0,17.0,5.0,4...|[0.24896720051239...|
|200002|    0|      21|         6|       387|45.0|       0.36|[387.0,21.0,6.0,4...|[0.35036475126652...|
|   125|    1|       0|         0|         8| 0.0|        0.0|       (5,[0],[8.0])|(5,[0],[0.0072426...|
|   124|    0|     171|        41|      4079|60.0|       2.83|[4079.0,171.0,41....|[3.69286258505468...|
|    51|    1|     100|        21|      2111|16.0|        5.5|[2111.0,100.0,21....|[1.91116276466056...|
|     7|    0|       7|         1|       150|51.0|       0.12|[150.0,7.0,1.0,51...|[0.13580029118857...|
|    15|    0|      81|        14|      1914|55.0|     

In [44]:
df_train, df_test = df_feature.randomSplit([0.8, 0.2],42)

In [45]:
lr = LogisticRegression(featuresCol = 'ScaledFeatures', labelCol = 'Churn', maxIter=10)
lrModel = lr.fit(df_train)

In [46]:
validation_results = lrModel.transform(df_valid)

In [47]:
validation_results.select('Churn','prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+



In [48]:
test_results = lrModel.transform(df_test)

In [49]:
test_results.select('Churn','prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [50]:
lr =  LogisticRegression(maxIter=10, regParam=0.0, elasticNetParam=0, featuresCol = 'ScaledFeatures', labelCol = 'Churn')
pipeline = Pipeline(stages=[lr])

In [51]:
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam,[0.0, 0.1]) \
    .build()

In [52]:
evaluator = MulticlassClassificationEvaluator(labelCol='Churn',
                                              predictionCol='prediction', metricName='f1') 

In [53]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

In [54]:
Model_lr = crossval.fit(df_train)

In [55]:
best_model = Model_lr.bestModel.stages[0]

In [56]:
print (best_model)

LogisticRegressionModel: uid = LogisticRegression_9eec29edf371, numClasses = 2, numFeatures = 5


In [57]:
test_results = best_model.transform(df_test)

In [58]:
evaluator.evaluate(test_results.select('prediction', 'Churn'))

0.7262656475019387

In [59]:
evaluator.evaluate(test_results.select('Churn', 'prediction'), {evaluator.metricName: "accuracy"})

0.7941176470588235

In [60]:
evaluator.metricName

Param(parent='MulticlassClassificationEvaluator_5b261a95b1bb', name='metricName', doc='metric name in evaluation (f1|weightedPrecision|weightedRecall|accuracy)')

In [61]:
type(Model_lr)

pyspark.ml.tuning.CrossValidatorModel

In [62]:
crossval.fit?

In [63]:
test_results.select('prediction', 'Churn').toPandas()

,prediction,Churn
0,0.0,0
1,1.0,1
2,0.0,0
3,0.0,1
4,0.0,0
5,0.0,0
6,0.0,1
7,0.0,0
8,0.0,1
9,0.0,0


# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.